In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Load the CSV file
file_path = '/content/rents_munich.csv'
data = pd.read_csv(file_path)

# Clean the Description column
data['Region'] = data['Description'].apply(lambda x: x.split('|')[1].replace('München', '').strip() if '|' in x else 'Unknown')

# Clean the Price and qm² columns
data['Price'] = data['Price'].str.extract(r'(\d+\.?\d*)').astype(float)
data['qm2'] = data['qm2'].str.extract(r'(\d+\.?\d*)').astype(float)

# Calculate price per qm²
data['price_per_qm'] = data['Price'] / data['qm2']

# Drop rows with missing values
data.dropna(inplace=True)

# Include more features for better prediction
data['log_qm2'] = np.log(data['qm2'])
data['log_price'] = np.log(data['Price'])

# EDA: Descriptive statistics
print("Descriptive Statistics:\n", data.describe())

# EDA: Correlation matrix
corr_matrix = data[['Price', 'qm2', 'price_per_qm', 'log_qm2', 'log_price']].corr()
print("\nCorrelation Matrix:\n", corr_matrix)

# Visualize correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('Correlation Matrix', fontsize=16)
plt.show()

# EDA: Distribution of price_per_qm
plt.figure(figsize=(10, 6))
sns.histplot(data['price_per_qm'], bins=30, kde=True, color='skyblue')
plt.title('Distribution of Price per qm²', fontsize=16)
plt.xlabel('Price per qm² (€)', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.show()

# Check for linear trend between qm2 and price_per_qm
pearson_corr, _ = pearsonr(data['qm2'], data['price_per_qm'])
print(f"Pearson Correlation between qm2 and price_per_qm: {pearson_corr}")

# Scatter plot to visualize the relationship
plt.figure(figsize=(10, 6))
sns.scatterplot(x='qm2', y='price_per_qm', data=data, color='purple', alpha=0.6)
plt.title('Price per qm² vs. qm²', fontsize=16)
plt.xlabel('qm²', fontsize=14)
plt.ylabel('Price per qm² (€)', fontsize=14)
plt.show()

# Split data into training and testing sets
X = data[['qm2', 'log_qm2', 'log_price']]
y = data['price_per_qm']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter tuning for Gradient Boosting Regressor
param_grid = {
    'n_estimators': [300, 400, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

gbr = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_gbr = grid_search.best_estimator_

# Train the best model
best_gbr.fit(X_train, y_train)

# Make predictions
y_pred = best_gbr.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Visualization of actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='green', alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', lw=2)
plt.xlabel('Actual', fontsize=14)
plt.ylabel('Predicted', fontsize=14)
plt.title('Actual vs Predicted Prices per qm²', fontsize=16)
plt.show()

# Group by region + calculate the mean price per qm²
region_price = data.groupby('Region')['price_per_qm'].mean().reset_index()

# Find the regions with the most expensive and cheapest apartments
most_expensive_region = region_price.loc[region_price['price_per_qm'].idxmax()]
cheapest_region = region_price.loc[region_price['price_per_qm'].idxmin()]

# Find the median price per qm²
median_price = region_price['price_per_qm'].median()
median_region = region_price.loc[(region_price['price_per_qm'] - median_price).abs().idxmin()]

# Find the three most expensive and three cheapest regions
most_expensive_regions = region_price.nlargest(3, 'price_per_qm')
cheapest_regions = region_price.nsmallest(3, 'price_per_qm')

# Color adjustment
red_palette = sns.color_palette('Reds', n_colors=3)
blue_palette = sns.color_palette('Blues', n_colors=3)

# Plot: regions by average price per qm²
plt.figure(figsize=(14, 6))
sns.barplot(data=region_price, x='Region', y='price_per_qm', palette='coolwarm', order=region_price.sort_values('price_per_qm')['Region'], ci=None)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.xlabel('Region', fontsize=12)
plt.ylabel('Price per qm² (€)', fontsize=12)
plt.title('Average Price per qm² by Region in Munich', fontsize=16)
plt.grid(False)
plt.tight_layout()

plt.subplots_adjust(bottom=0.3)

# Plot: most expensive and cheapest regions
plt.figure(figsize=(8, 4))
sns.barplot(x=[most_expensive_region['Region'], cheapest_region['Region']],
            y=[most_expensive_region['price_per_qm'], cheapest_region['price_per_qm']],
            palette=[red_palette[0], blue_palette[0]], ci=None)
plt.title('Price per qm² in the Most Expensive and Cheapest Regions', fontsize=16)
plt.xlabel('Region', fontsize=14)
plt.ylabel('Price per qm² (€)', fontsize=14)
plt.grid(False)
plt.tight_layout()

# Prices placed horizontally next to the bars
for i, val in enumerate([most_expensive_region['price_per_qm'], cheapest_region['price_per_qm']]):
    plt.text(i, val, f'{val:.2f}', ha='center', va='bottom', fontsize=12, color='black')

# Chart: the three most expensive and cheapest regions
plt.figure(figsize=(12, 6))
sns.barplot(data=pd.concat([most_expensive_regions, cheapest_regions]), x='Region', y='price_per_qm', hue='Region', dodge=False, ci=None, palette=[red_palette[0], red_palette[1], red_palette[2], blue_palette[0], blue_palette[1], blue_palette[2]])
plt.title('Three Most Expensive and Cheapest Regions', fontsize=16)
plt.xlabel('Region', fontsize=14)
plt.ylabel('Price per qm² (€)', fontsize=14)
plt.grid(False)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.tight_layout()

plt.show()